In [5]:
import os
import json
import random
import time
import sys

# 플레이어 클래스
class Player:
    def __init__(self):
        self.health = 100
        self.mental = 100
        self.money = 50
        self.inventory = []

    def update_stats(self, effect):
        self.health += effect.get("health", 0)
        self.mental += effect.get("mental", 0)
        self.money += effect.get("money", 0)
        item = effect.get("item")
        if item:
            self.inventory.append(item)
            print(f"\n아이템 '{item}'을(를) 획득했습니다.")
        self.print_status()

    def consume_item(self, item_name):
        if item_name in self.inventory:
            self.inventory.remove(item_name)
            print(f"\n아이템 '{item_name}'을(를) 사용하여 소모했습니다.")
        else:
            print(f"\n아이템 '{item_name}'이(가) 인벤토리에 없습니다.")

    def is_game_over(self):
        return self.health <= 0 or self.mental <= 0

    def print_status(self):
        print("-"*40)
        print(f"현재 상태 - 체력: {self.health}, 멘탈: {self.mental}, 돈: {self.money}")
        print(f"인벤토리: {self.inventory}\n")

\
# JSON 파일 로드 함수
def load_encounter(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        return json.load(f)


# 선택지 조건 확인 함수
def is_choice_available(player, requirements):
    if not requirements:
        return True
    if "min_health" in requirements and player.health < requirements["min_health"]:
        return False
    if "min_mental" in requirements and player.mental < requirements["min_mental"]:
        return False
    if "min_money" in requirements and player.money < requirements["min_money"]:
        return False
    if "requires_item" in requirements:
        if requirements["requires_item"] not in player.inventory:
            return False
    return True

# 게임 즉시 종료 함수
def get_input(prompt):
    user_in = input(prompt)
    if user_in.lower() == 'x':
        print("\nx 입력으로 게임을 종료합니다.")
        sys.exit(0)
    return user_in

# 단일 인카운터 진행 함수 (단계가 없는 경우)
def run_encounter(player, encounter_data):
    print("\n" + "="*40)
    print(encounter_data.get("story", ""))
    print("="*40)

    available_choices = []
    for idx, choice in enumerate(encounter_data["choices"]):
        req = choice.get("requirements", {})
        if is_choice_available(player, req):
            available_choices.append((idx, choice))
            print(f"{idx + 1}. {choice['description']}")
        else:
            print(f"{idx + 1}. {choice['description']} (선택 불가)")

    while True:
        try:
            choice_input = int(get_input("번호를 선택하세요: ")) - 1
            if any(choice_input == idx for idx, _ in available_choices):
                break
            else:
                print("해당 선택지는 사용할 수 없습니다. 다시 선택해주세요.")
        except ValueError:
            print("숫자를 입력해주세요.")

    chosen_choice = encounter_data["choices"][choice_input]
    print("\n" + chosen_choice["player_response"])

    if "consume_item" in chosen_choice:
        player.consume_item(chosen_choice["consume_item"])

    player.update_stats(chosen_choice["effect"])


# 연속된 스토리(단계)를 가진 인카운터 진행 함수
def run_continuous_encounter(player, encounter_data):
    stages = encounter_data.get("stages", [])
    for stage in stages:
        print("\n" + "="*40)
        print(stage.get("story", ""))
        print("="*40)

        available_choices = []
        for idx, choice in enumerate(stage["choices"]):
            req = choice.get("requirements", {})
            if is_choice_available(player, req):
                available_choices.append((idx, choice))
                print(f"{idx + 1}. {choice['description']}")
            else:
                print(f"{idx + 1}. {choice['description']} (선택 불가)")

        if not available_choices:
            print("사용 가능한 선택지가 없습니다. 이 단계를 건너뜁니다.")
            continue

        while True:
            try:
                choice_input = int(get_input("번호를 선택하세요: ")) - 1
                if any(choice_input == idx for idx, _ in available_choices):
                    break
                else:
                    print("해당 선택지는 사용할 수 없습니다. 다시 선택해주세요.")
            except ValueError:
                print("숫자를 입력해주세요.")

        chosen_choice = stage["choices"][choice_input]
        print("\n" + chosen_choice["player_response"])

        if "consume_item" in chosen_choice:
            player.consume_item(chosen_choice["consume_item"])

        player.update_stats(chosen_choice["effect"])

        if player.is_game_over():
            print("체력이나 멘탈이 0 이하가 되어 게임 오버입니다!")
            return  # 게임 종료

In [6]:
# introduction : 게임 배경 소개 및 튜토리얼얼

def introduction(player):
    print("\n안녕하세요, 저는 이야기꾼이라고 합니다.")
    time.sleep(1)
    print("이 이야기가 어떤 결과로 흘러 갈 지는 당신의 행동과 판단이 결정하게 됩니다.")
    time.sleep(1)
    print("저는 그저 이야기를 전달할 뿐, 모든 고민과 결정은 당신이 수행해야 합니다.")
    time.sleep(1.5)
    
    # 튜토리얼 건너뛰기 여부 입력 받기 (x 입력 시 종료 기능 등과 연동할 수 있음)
    print("\n이 게임을 플레이 해 본 적 있으신가요? (예/아니오, 예를 누르면 튜토리얼을 건너뜁니다)")
    print("="*40)
    print("1. 예, 다 아는 이야기네요 \n2. 아니요! 구체적인 설명이 필요합니다.")
    
    while True:
        answer = input()
        if answer.strip() == "1":
            print("\n튜토리얼을 건너뛰겠습니다.\n")
            break
        elif answer.strip() == "2":
            print("\n2033, 서울은 핵전쟁으로 인해 폐허가 되었습니다.")
            time.sleep(1)
            print("\n당신은 도봉구 밑 작은 마을, 무너진 집에서 눈을 떴습니다.")
            time.sleep(1)
            print("\n구현 중... (중요한거 아니니깐 ㅎㅎㅎ)")
            time.sleep(1)
            print("\n게임을 종료하고 싶을 때, x키를 눌러 즉시 종료시킬 수 있습니다.")
            time.sleep(1)
            print("\n튜토리얼은 여기까지입니다! 앞으로는 직접 경험하고 익혀야겠지요.\n")
            break
        else :
            print("\n잘못된 입력 입니다. (숫자를 입력해주세요.)")
    
    time.sleep(1)
    
    # 시작 아이템 뽑기
    starting_items = ["민첩함", "신중함", "집중력"]
    item = random.choice(starting_items)
    player.inventory.append(item)
    print("="*40)
    
    # 각 아이템에 따른 설명 출력
    if item == "민첩함":
        print("고등학교 시절 운동부였던 당신은 남들보다 민첩하게 움직일 수 있습니다. \n위급한 상황에서도 빠르게 대응할 수 있겠지요.")
        print("="*40)
    elif item == "신중함":
        print("평소 신중한 성격 덕분에, 중요한 결정을 내릴 때 깊이 생각하며 행동합니다.")
        print("="*40)
    elif item == "집중력":
        print("탁월한 집중력을 가진 당신은 한 번 시작한 일은 끝까지 해내는 끈기가 있습니다.")
        print("="*40)
    
    time.sleep(1)
    print("\n그럼 이제 당신의 여정을 떠나야 할 시간입니다.")
    time.sleep(1)
    print("\n행운을 빕니다...")
    time.sleep(1)
    player.print_status()

In [7]:
# 인카운터 파일 목록 로드 함수
def load_encounter_list(directory):
    files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".json")]
    return files

# 게임 루프 (사이클 반복: 랜덤 인카운터 최소 3회 후 남은 메인 인카운터 등장)
def game_loop(player):
    random_encounter_dir = "./random_encounters"
    main_encounter_dir = "./main_encounters"
    random_encounters = load_encounter_list(random_encounter_dir)
    available_main_encounters = load_encounter_list(main_encounter_dir)  # 메인 인카운터는 한 번씩만 등장
    previous_random_encounter = None
    MIN_RANDOM_ENCOUNTERS = 3

    while True:
        random_count = 0
        # 최소 MIN_RANDOM_ENCOUNTERS 만큼 랜덤 인카운터 진행
        while random_count < MIN_RANDOM_ENCOUNTERS:
            encounter_file = random.choice(random_encounters)
            # 직전 인카운터와 동일하면 건너뜀
            if encounter_file == previous_random_encounter and len(random_encounters) > 1:
                continue
            encounter_data = load_encounter(encounter_file)
            run_encounter(player, encounter_data)
            if player.is_game_over():
                return
            previous_random_encounter = encounter_file
            random_count += 1

        # 남은 메인 인카운터가 있다면 등장
        if available_main_encounters:
            main_encounter_file = random.choice(available_main_encounters)
            available_main_encounters.remove(main_encounter_file)
            main_encounter_data = load_encounter(main_encounter_file)
            if "stages" in main_encounter_data:
                run_continuous_encounter(player, main_encounter_data)
            else:
                run_encounter(player, main_encounter_data)
            if player.is_game_over():
                return

In [8]:
def main():
    player = Player()
    introduction(player)
    game_loop(player)
    time.sleep(0.5)
    print("\n당신은 최선을 다하였지만, 더 이상 나아갈 수 없습니다...")
    time.sleep(0.5)
    print("\n하지만 뜻 깊은 기록을 남기셨기를 바랍니다.")
    time.sleep(1)
    

if __name__ == "__main__":
    while True:
        main()
        print("\n게임이 종료되었습니다. 다시 플레이하시겠습니까? (y/n)")
        replay = input()
        if replay.lower() != 'y':
            print("\n플레이해주셔서 감사합니다. 게임을 종료합니다.")
            break


안녕하세요, 저는 이야기꾼이라고 합니다.
이 이야기가 어떤 결과로 흘러 갈 지는 당신의 행동과 판단이 결정하게 됩니다.
저는 그저 이야기를 전달할 뿐, 모든 고민과 결정은 당신이 수행해야 합니다.

이 게임을 플레이 해 본 적 있으신가요? (예/아니오, 예를 누르면 튜토리얼을 건너뜁니다)
1. 예, 다 아는 이야기네요 
2. 아니요! 구체적인 설명이 필요합니다.

튜토리얼을 건너뛰겠습니다.

탁월한 집중력을 가진 당신은 한 번 시작한 일은 끝까지 해내는 끈기가 있습니다.

그럼 이제 당신의 여정을 떠나야 할 시간입니다.

행운을 빕니다...
----------------------------------------
현재 상태 - 체력: 100, 멘탈: 100, 돈: 50
인벤토리: ['집중력']


골라 골라~ 당신은 동대문 시장에 도착했습니다.
구매할 만 한 것이 있으려나?
1. 열쇠
2. 롱패딩
3. 식량
4. 응급구조키트

추운건 싫어...

아이템 '롱패딩'을(를) 획득했습니다.
----------------------------------------
현재 상태 - 체력: 100, 멘탈: 100, 돈: 40
인벤토리: ['집중력', '롱패딩']


당신은 전쟁 전에 짓다 만 도서관을 발견했습니다.
무슨 책을 읽을까요?
1. 왕초보 생존 영어 회화
2. 셜록홈즈
3. 타짜의 기술
4. 내일은 유도왕

동작 그만, 맡장 빼기냐? (멘탈-5, 언변)

아이템 '언변'을(를) 획득했습니다.
----------------------------------------
현재 상태 - 체력: 100, 멘탈: 95, 돈: 40
인벤토리: ['집중력', '롱패딩', '언변']


식사 시간 입니다!
뭔가를 좀 먹어야 겠습니다.
1. 식사를 한다. (선택 불가)
2. 사냥을 좀 해볼까
3. 굶는다.

음.. 조금 찝찝한걸 (체력 +5, 멘탈 +5)
----------------------------------------
현재 상태 - 체력: 105, 멘탈: 100, 돈: 4

KeyError: 'effect'